In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px
import io
from datetime import timedelta
from pandas.tseries.offsets import CustomBusinessDay

In [2]:
path_faturados = r'D:\Users\Vinicius Santos\OneDrive\SERVIDOR\FATURAMENTO-\05 - FATURAMENTO GERAL\2023 Banco de Emissões.xlsm'
df_faturados = pd.read_excel(path_faturados, header=1, dtype={'Série': str, 'Nº Doc': str, 'NFS': str})
df_faturados['Doc'] = np.where(df_faturados['Série'] == '1', df_faturados['NFS'], df_faturados['Nº Doc']).astype(str)
df_faturados['Série'] = np.where(df_faturados['Série'] == '1', '0', df_faturados['Série']).astype(str)
df_faturados['Entrega'] = np.where(df_faturados['Recebedor'].isna() | (df_faturados['Recebedor'] == ''),
                                   df_faturados['Destinatário'], df_faturados['Recebedor'])
df_faturados['Mun_Entrega'] = np.where(df_faturados['Recebedor'].isna() | (df_faturados['Recebedor'] == ''),
                                       df_faturados['Dest. Cidade'], df_faturados['Rec. Cidade'])

In [3]:
path_pagos = r'D:\Users\Vinicius Santos\OneDrive\SERVIDOR\PAGAMENTO\2024-DOCUMENTOS PAGOS.xlsm'
df_pagos = pd.read_excel(path_pagos, header=1, dtype={'Serie': str, 'CT-e/NFS': str})

In [4]:
df_pagamento = pd.merge(df_faturados, df_pagos, left_on=['Série','Doc'], right_on=['Serie','CT-e/NFS'], how='left')

In [5]:
def calcular_vencimento_sem_feriados(data_emissao):
    """
    Calcula a data de pagamento com prazo de 60 dias após a emissão,
    programado para o 1º dia útil do mês subsequente (sem considerar feriados).
    """
    # Calcula a data de vencimento inicial (60 dias após a emissão)
    vencimento_inicial = data_emissao + timedelta(days=60)

    # Calcula o primeiro dia do mês seguinte ao vencimento inicial
    proximo_mes = vencimento_inicial.month + 1
    proximo_ano = vencimento_inicial.year
    if proximo_mes > 12:
        proximo_mes = 1
        proximo_ano += 1
    primeiro_dia_proximo_mes = pd.Timestamp(f'{proximo_ano}-{proximo_mes}-01')

    # Ajusta para o primeiro dia útil (ignorando feriados)
    data_pagamento = primeiro_dia_proximo_mes
    while data_pagamento.weekday() >= 5:  # 5 = Sábado, 6 = Domingo
        data_pagamento += timedelta(days=1)
    return data_pagamento

# Supondo que você já tenha o DataFrame df_faturamento com a coluna 'Emissao'
df_pagamento['Vencimento'] = df_pagamento['Dt. Emissão'].apply(calcular_vencimento_sem_feriados)

In [6]:
df_pagamento['Status_Pagamento'] = np.where(
    df_pagamento['Valor Pago'].isna() & (pd.Timestamp.now() < df_pagamento['Vencimento']),
    'Pendente',
    np.where(df_pagamento['Valor Pago'].isna(), "À vencer", "Pago"))

In [7]:
df_pagamento['Valor Pago'] = df_pagamento['Valor Pago'].fillna(0)
df_pagamento['Saldo a receber'] = df_pagamento['Total'] - df_pagamento['Valor Pago']

In [8]:
df_pagamento = df_pagamento[['Doc', 'Dt. Emissão', 'NFe', 'Remetente', 'Rem. Cidade',
       'Destinatário', 'Dest. Cidade', 'Recebedor', 'Rec. Cidade', 'Total',
       'Frete', 'Pedágio', 'Peso Bruto', 'Observação',
       'Placa', 'Status', 'Tomador', 'Fatura', 'Entrega', 'Mun_Entrega', 'Serie', 'Vencimento',
       'Valor Pago', 'Dt. Repasse', 'Status_Pagamento','Saldo a receber']]
display(df_pagamento)

,Doc,Dt. Emissão,NFe,Remetente,Rem. Cidade,Destinatário,Dest. Cidade,Recebedor,Rec. Cidade,Total,...,Tomador,Fatura,Entrega,Mun_Entrega,Serie,Vencimento,Valor Pago,Dt. Repasse,Status_Pagamento,Saldo a receber
0,39831,2023-01-02 09:29:39,"257770, 257773, 257772",BRASKEM S.A,Santo André,QUANTIQ DISTRIBUIDORA LTDA,Guarulhos,NaN,NaN,995.85,...,QUANTIQ DISTRIBUIDORA LTDA,NaN,QUANTIQ DISTRIBUIDORA LTDA,Guarulhos,NaN,2023-04-03,0.00,NaT,À vencer,995.85
1,39832,2023-01-02 10:07:26,660170,QUANTIQ DISTRIBUIDORA LTDA,Guarulhos,QUAKER CHEMICAL INDUSTRIA E COMERCI LTDA,Mauá,NaN,NaN,985.08,...,QUANTIQ DISTRIBUIDORA LTDA,NaN,QUAKER CHEMICAL INDUSTRIA E COMERCI LTDA,Mauá,NaN,2023-04-03,0.00,NaT,À vencer,985.08
2,39833,2023-01-02 10:07:27,"660172, 660171",QUANTIQ DISTRIBUIDORA LTDA,Guarulhos,SIEGWERK BRASIL INDUSTRIA DE TINTAS,Diadema,NaN,NaN,202.50,...,QUANTIQ DISTRIBUIDORA LTDA,NaN,SIEGWERK BRASIL INDUSTRIA DE TINTAS,Diadema,NaN,2023-04-03,0.00,NaT,À vencer,202.50
3,39834,2023-01-02 10:25:10,"660159, 660158",QUANTIQ DISTRIBUIDORA LTDA,Guarulhos,ECOMASTER QUIMICA INDUSTRIA E COMERCIO LTDA,Uberaba,TRANSCAJURU LTDA,São Paulo,125.91,...,QUANTIQ DISTRIBUIDORA LTDA,NaN,TRANSCAJURU LTDA,São Paulo,NaN,2023-04-03,0.00,NaT,À vencer,125.91
4,39835,2023-01-02 13:23:00,660169,QUANTIQ DISTRIBUIDORA LTDA,Guarulhos,VMAX COMERCIO DE PRODUTOS DE LIMPEZA EIRELLI EPP,São Paulo,NaN,NaN,124.03,...,QUANTIQ DISTRIBUIDORA LTDA,NaN,VMAX COMERCIO DE PRODUTOS DE LIMPEZA EIRELLI EPP,São Paulo,NaN,2023-04-03,0.00,NaT,À vencer,124.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17256,23238,2025-02-24 15:23:03,6319,IQAG ARMAZENS GERAIS LTDA.,Guarulhos,ADITIVE IND. E COM. DE PRODUTOS PARA CONSTRUCA...,Guaratinguetá,JOYCE MARIANA FERMI DE PAULA MUNOZ 32620779847,Guarulhos,287.51,...,AODRAN DO BRASIL COM DE PRODUTOS QUIMICOS LTDA,759.0,JOYCE MARIANA FERMI DE PAULA MUNOZ 32620779847,Guarulhos,0,2025-05-01,287.51,2025-03-30,Pago,0.00
17257,23239,2025-02-26 15:14:06,6329,IQAG ARMAZENS GERAIS LTDA.,Guarulhos,ADITIVE IND. E COM. DE PRODUTOS PARA CONSTRUCA...,Guaratinguetá,JOYCE MARIANA FERMI DE PAULA MUNOZ 32620779847,Guarulhos,287.51,...,AODRAN DO BRASIL COM DE PRODUTOS QUIMICOS LTDA,759.0,JOYCE MARIANA FERMI DE PAULA MUNOZ 32620779847,Guarulhos,0,2025-05-01,287.51,2025-03-30,Pago,0.00
17258,23240,2025-02-26 15:14:07,6325,IQAG ARMAZENS GERAIS LTDA.,Guarulhos,CHEMISEAL INDUSTRIA DE PRODUTOS PARA VEDACAO L...,Guarulhos,CHEMISEAL INDUSTRIA DE PRODUTOS PARA VEDACAO L...,Guarulhos,291.50,...,AODRAN DO BRASIL COM DE PRODUTOS QUIMICOS LTDA,759.0,CHEMISEAL INDUSTRIA DE PRODUTOS PARA VEDACAO L...,Guarulhos,0,2025-05-01,291.50,2025-03-30,Pago,0.00
17259,23241,2025-02-27 14:09:02,307,IQAG ARMAZENS GERAIS LTDA.,Guarulhos,OTIMIZA IND E COMDE PRODUTOS LTDA,Guarulhos,OTIMIZA IND E COMDE PRODUTOS LTDA,Guarulhos,307.63,...,ABACC CHEMICAL IMPORTACAO E EXPORTACAO LTDA,760.0,OTIMIZA IND E COMDE PRODUTOS LTDA,Guarulhos,NaN,2025-05-01,0.00,NaT,Pendente,307.63
